In [1]:
from collections import defaultdict, deque
import random
import math
import time

import numpy as np

class City:
    id = 0
    def __init__(self):
        self.X = random.randint(-100, 100)
        self.Y = random.randint(-100, 100)
        self.Z = random.randint(0, 50)
        self.coordinates = np.array((self.X, self.Y, self.Z))
        self.id = City.id
        City.id += 1
    
class Graph:
    
    def __init__(self, cities):
        self.all_paths = []
        self.distance_list = []
        self.adjList = defaultdict(list)
        self.sym_matrix = self.generate_sym_matrix(cities)
        self.matrix_80 = self.generate_80_matrix(cities)
        self.asym100_matrix = self.generate_asym_matrix(cities)
        self.asym80_matrix = self.generate_80asym_matrix(cities)

    #Stworzenie symetrycznej macierzy 100%

    def generate_sym_matrix(self, cities):
        sym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            sym_matrix.append(row)
        return np.array(sym_matrix)
    

    #Stworzenie symetrycznej macierzy 80%

    def generate_80_matrix(self, cities):
        matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            matrix.append(row)
        matrix = np.array(matrix)
        total_elements = matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(matrix.shape)
    

    #Stworzenie asymetrycznej macierzy 100%

    def generate_asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        return np.array(asym_matrix)
    

    #Stworzenie asymetrycznej macierzy 80%

    def generate_80asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        asym_matrix = np.array(asym_matrix)
        total_elements = asym_matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = asym_matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1 or matrix_flattened[index] == 2:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(asym_matrix.shape)


    #Metoda obliczajaca odleglosc bez uwzglednienia asymetrycznosci
    def calculate_distance(self, city1, city2):
        return np.linalg.norm(city1.coordinates - city2.coordinates)
    


    #Metoda obliczajaca odleglosc z uwzglednieniem asymetrycznosci
    def calculate_distance_asym(self, city1, city2):
        if city1.Z < city2.Z:
            #print("Going up")
            return 1.1 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        elif city1.Z > city2.Z:
            #print("Going down")
            return 0.9 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        else:
            #print("Same height")
            return np.linalg.norm(city1.coordinates - city2.coordinates)


    #Dodawanie krawedzi do grafu (do zmiennej adjList)
    def addEdge(self, u, v):
            self.adjList[u].append(v)

    #Definiowanie krawedzi i przekazanie do metody addEdge
    def createEdge(self, matrix):
        for i in range(0, len(matrix)):
            for j in range(0, len(matrix)):
                if matrix[i][j] == 1 or matrix[i][j] == 2:
                    self.addEdge(i, j)
                    #print(f"Created edge between {i} and {j}")


    def calculate_distance_from_path(self, path, cities, sym):
        total_distance = 0
        for i in range(0, len(path) - 1):
            if sym == 'sym':
                total_distance += self.calculate_distance(cities[path[i]], cities[path[i + 1]])
            elif sym == 'asym':
                total_distance += self.calculate_distance_asym(cities[path[i]], cities[path[i + 1]])
        return total_distance

    def ACO(self, cities, method):
        epochs = 100
        fx = len(cities)
        ants = 50 * fx
        f = np.array([0] * ants)
        ants_paths = []
        

        ants_paths = self.initial_scan(ants, ants_paths, fx, f)

        for _ in range(epochs):
            ants_paths, prob = self.epoch(ants_paths, f, fx, cities, method)

        prob = prob.index(max(prob))
        return ants_paths[prob], self.calculate_distance_from_path(ants_paths[prob], cities, method)
    
    def initial_scan(self, ants, ants_path, fx, f):
        currentNode = 0
        for _ in range(ants):
            # print(f'Ant {ant}')
            visited = set()
            visited.add(0)
            currentNode = 0
            path = [0] #* ants
            # print(ants_path)
            # print(path[i])
            i = 0
            
            # Warunek ktory sprawdza czy dlugosc nie jest wystarczajaca oraz czy nie przekroczono liczby iteracji
            while len(path) < fx and i < (3 * fx):
                # Warunek na dodanie koncowego powrotu do miasta 0 jezeli to mozliwe
                if len(path) == (fx - 1) and neighbour == 0:
                    path.append(neighbour)
                    break
                
                # Sprawdzenie czy aktualne miasto ma jakiegos sasiada oraz czy liczba sasiadow jest rowna dlugosci odwiedzonych
                if len(self.adjList[currentNode]) != 0 or len(self.adjList[currentNode]) != len(visited):
                    # print(f'Adj: {self.adjList[currentNode]}, Visted: {visited}')
                    # neighbour = np.random.choice(self.adjList[currentNode], p=[0.1, 0.1, 0.1, 0.7])

                    # Wybranie losowego sasiada
                    neighbour = np.random.choice(self.adjList[currentNode])


                    # print(f'Current node: {currentNode}, current neighoburs {self.adjList[currentNode]}, choice: {neighbour}')
                    # neighbour = self.adjList[currentNode][random_choice]
                    # print(f'Neighbour: {neighbour}')
                else: break
            
                # Dodawanie sasiada pod warunkiem, ze nie byl on juz odwiedzony
                if neighbour not in path:
                    path.append(neighbour)
                    # print(f'Neighbour node before: {neighbour})')
                    currentNode = neighbour
                    visited.add(neighbour)
                    # print(f'Current node after: {currentNode})')
                    # print(path)
                i += 1
            # print(f'Path: {path}')
            # Warunek sprawdzajacy czy dlugosc sciezki odpowiada ustalonej dlugosci
            if len(path) == fx:
                ants_path.append(path)
            # else:
                # print(f'\033[92mBad path: {path}\033[0m')
                

        return ants_path


    # Pojedyncza epoka
    def epoch(self, ants_paths, f, fx, cities, method):
        path_quantity = defaultdict(lambda: 0)
        probability = []
        sum_f = 0
        new_ants_paths = []
        new_probability = []

        for path in ants_paths:
            # Zliczane ile razy poszczegolna sciezka byla wybrana przez mrowki
            path_quantity[tuple(path)] += 1
            # print(path_quantity.keys())

        for key in path_quantity:
            # Obliczenie dlugosci sciezki
            dist = self.calculate_distance_from_path(key, cities, method)
            # Zapisanie ilosci odbytych sciezek przez mrowki
            quantity = path_quantity[key]
            # Dodanie do poszczegolnej sciezki odpowiednio: quanity, stosunek feromonu do dystansu pomnozony przez quanity
            path_quantity[key] = [quantity, dist, (fx/dist) * quantity * 100]
            # Sumowanie zostawionego feromonu [(fx/dist) * quantity * 100]
            sum_f += path_quantity[key][2]
        
        for i, path in enumerate(path_quantity):
            # Obliczenie szansy na wybranie danej sciezki ktora jest stosunkiem zostawionego feromonu do ich sumy
            probability.append(path_quantity[path][2]/sum_f)
            # print(f'Prob {i}: {probability[i]}')
        # print('\n')
        for _ in range(len(ants_paths)):
            # Wybranie sciezki bazujac jedynie na prawdopodobienstwu
            choice = np.random.choice(len(list(path_quantity.keys())), p=probability)
            # print(f'{choice}: {ants_paths[choice]}')
            # Dodawanie do list sciezkek oraz ich prawdopodobienstwa
            new_ants_paths.append(ants_paths[choice])
            new_probability.append(probability[choice])
        # print(f'\nNew prob: {new_probability}')
        return new_ants_paths, probability

In [16]:
cities = []
for i in range(0, 11):
    cities.append(City())
    # print(f'City {i}: X: {cities[i].X}, Y: {cities[i].Y}, Z: {cities[i].Z}')
graph = Graph(cities)

#print(graph.sym_matrix)
#print(graph.matrix_80)
# print(graph.asym100_matrix)
#print(graph.asym80_matrix)

graph.createEdge(graph.sym_matrix)
# graph.createEdge(graph.matrix_80)
# graph.createEdge(graph.asym100_matrix)
# graph.createEdge(graph.asym80_matrix)

print('\nACO')
start_time = time.perf_counter()
ACO_path, ACO_dist = graph.ACO(cities, 'sym')
print(f'Best path: {ACO_path}, distance: {ACO_dist}')
end_time = time.perf_counter()
print(f'ACO time: {(end_time - start_time)}')


ACO
Best path: [0, 1, 5, 2, 8, 9, 3, 6, 7, 4, 0], distance: 774.3621432135158
ACO time: 0.27885809999997946
